In [ ]:
!git clone https://github.com/huggingface/diffusers

Cloning into 'diffusers'...
remote: Enumerating objects: 112237, done.
remote: Counting objects: 100% (373/373), done.
remote: Compressing objects: 100% (247/247), done.
remote: Total 112237 (delta 274), reused 126 (delta 126), pack-reused 111864 (from 3)
Receiving objects: 100% (112237/112237), 86.00 MiB | 30.66 MiB/s, done.
Resolving deltas: 100% (83631/83631), done.


In [ ]:
%cd diffusers
!pip install .

/content/diffusers
Processing /content/diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.36.0.dev0-py3-none-any.whl size=4521797 sha256=fedfffa34002627fc69fb35f447ab7dd815c3a780e8d742733f7d499f616cd94
  Stored in directory: /tmp/pip-ephem-wheel-cache-nq8menvi/wheels/8a/fc/09/385efb77b455b2fd4a656c950079c93147e1f50ae614e51beb
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.35.2
    Uninstalling diffusers-0.35.2:
      Successfully uninstalled diffusers-0.35.2


In [ ]:
%cd examples/instruct_pix2pix
!pip install -r requirements.txt

/content/diffusers/examples/instruct_pix2pix
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00


In [ ]:
!accelerate config default

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ============================================================================
# TRAINING 1: Pretrained Model (from stable-diffusion-v1-5)
# This will be used as the "background/prior" model in guidance mechanism
# ============================================================================

!accelerate launch --mixed_precision="fp16" train_instruct_pix2pix.py \
    --pretrained_model_name_or_path="stable-diffusion-v1-5/stable-diffusion-v1-5"  \
    --dataset_name="fusing/instructpix2pix-1000-samples" \
    --resolution=256 \
    --random_flip \
    --train_batch_size=4 \
    --gradient_accumulation_steps=4 \
    --gradient_checkpointing \
    --max_train_steps=250 \
    --checkpointing_steps=5000 \
    --checkpoints_total_limit=1 \
    --learning_rate=5e-05 \
    --max_grad_norm=1 \
    --lr_warmup_steps=0 \
    --conditioning_dropout_prob=0.05 \
    --mixed_precision=fp16 \
    --output_dir="/content/drive/MyDrive/genai/instructpix2pix/pretrained_model" \
    --seed=42

2025-11-27 01:56:56.531639: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-27 01:56:56.549087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764208616.570782    4037 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764208616.577385    4037 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764208616.593945    4037 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
# ============================================================================
# TRAINING 2: Trained Model (from customized fine-tuned Stable Diffusion)
# This will be used as the "fine-tuned" model in guidance mechanism
# ============================================================================

# IMPORTANT: Replace this path with your customized fine-tuned Stable Diffusion model
# This should be a standard Stable Diffusion model (4-channel UNet) that you've fine-tuned
# Examples:
#   - A DreamBooth model: "path/to/your/dreambooth-model"
#   - A LoRA model converted to full weights: "path/to/your/lora-model"
#   - Any other fine-tuned SD v1.5 model

# IMPORTANT: Replace this path with your customized fine-tuned Stable Diffusion model
CUSTOMIZED_SD_MODEL_PATH = "stable-diffusion-v1-5/stable-diffusion-v1-5"  # TODO: Replace with your customized model path

print(f"Training InstructPix2Pix model from customized SD model: {CUSTOMIZED_SD_MODEL_PATH}")
print("Note: The training script will automatically convert it to InstructPix2Pix format (8-channel UNet)")

# Build the command with the custom model path
train_cmd = f'''accelerate launch --mixed_precision="fp16" train_instruct_pix2pix.py \\
    --pretrained_model_name_or_path="{CUSTOMIZED_SD_MODEL_PATH}"  \\
    --dataset_name="fusing/instructpix2pix-1000-samples" \\
    --resolution=256 \\
    --random_flip \\
    --train_batch_size=4 \\
    --gradient_accumulation_steps=4 \\
    --gradient_checkpointing \\
    --max_train_steps=250 \\
    --checkpointing_steps=5000 \\
    --checkpoints_total_limit=1 \\
    --learning_rate=5e-05 \\
    --max_grad_norm=1 \\
    --lr_warmup_steps=0 \\
    --conditioning_dropout_prob=0.05 \\
    --mixed_precision=fp16 \\
    --output_dir="/content/drive/MyDrive/genai/instructpix2pix/trained_model" \\
    --seed=42'''

!{train_cmd}

print("\n✓ Trained model training complete!")
print("Saved to: /content/drive/MyDrive/genai/instructpix2pix/trained_model")


In [ ]:
import PIL
import requests
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline
from diffusers.utils import load_image

pipeline = StableDiffusionInstructPix2PixPipeline.from_pretrained("/content/drive/MyDrive/genai/instructpix2pix/model", torch_dtype=torch.float16).to("cuda")
generator = torch.Generator("cuda").manual_seed(0)

image = load_image("https://huggingface.co/datasets/sayakpaul/sample-datasets/resolve/main/test_pix2pix_4.png")
prompt = "add duck on the lake"
num_inference_steps = 20
image_guidance_scale = 2
guidance_scale = 8

edited_image = pipeline(
   prompt,
   image=image,
   num_inference_steps=num_inference_steps,
   image_guidance_scale=image_guidance_scale,
   guidance_scale=guidance_scale,
   generator=generator,
).images[0]
edited_image.save("duck8_2.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# ============================================================================
# BACKGROUND GUIDANCE FOR INSTRUCTPIX2PIX
# Similar to bg_guidance.py but adapted for InstructPix2Pix pipeline
# MEMORY OPTIMIZED VERSION - includes attention slicing, VAE slicing, and cache clearing
# ============================================================================

import PIL
import requests
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline
from diffusers.utils import load_image
from tqdm.auto import tqdm
import numpy as np

# Clear any existing GPU cache
torch.cuda.empty_cache()


def generate_with_bg_guidance(
    pipeline_trained,
    pipeline_pretrained,
    prompt,
    image,
    num_inference_steps=20,
    image_guidance_scale=2.0,
    guidance_scale=8.0,
    omega=0.5,  # Interpolation weight: 0.0 = only pretrained, 1.0 = only trained
    generator=None,
):
    """
    Generate image with background guidance that blends predictions from
    pretrained and trained InstructPix2Pix models.

    This is similar to bg_guidance.py but adapted for InstructPix2Pix.
    At each denoising step, it blends the noise predictions from both models.
    
    MEMORY OPTIMIZED: This function includes memory optimizations:
    - Explicit tensor deletion and cache clearing
    - Sequential model processing
    - torch.no_grad() context
    - Assumes attention slicing and VAE slicing are enabled on pipelines

    Args:
        pipeline_trained: Fine-tuned InstructPix2Pix pipeline
        pipeline_pretrained: Pretrained InstructPix2Pix pipeline
        prompt: Edit instruction text
        image: Input image to edit (PIL Image or tensor)
        num_inference_steps: Number of denoising steps (reduce if OOM)
        image_guidance_scale: Image guidance scale (for InstructPix2Pix)
        guidance_scale: Text guidance scale (classifier-free guidance)
        omega: Interpolation weight between models (0.0-1.0)
               - 0.0: Use only pretrained model (more general, preserves background)
               - 1.0: Use only trained model (more specific to training data)
               - 0.5: Equal blend of both models
        generator: Random generator for reproducibility
    
    Returns:
        PIL Image: Generated edited image
    
    Note: If you still get OOM errors, try:
    - Reducing image resolution (resize input image before calling)
    - Reducing num_inference_steps
    - Enabling CPU offloading on pipelines
    - Processing with smaller batch sizes
    """
    device = pipeline_trained.device
    dtype = next(pipeline_trained.unet.parameters()).dtype
    
    # 1. Preprocess image
    if isinstance(image, PIL.Image.Image):
        image = pipeline_trained.image_processor.preprocess(image).to(device=device, dtype=dtype)
    elif isinstance(image, torch.Tensor):
        image = image.to(device=device, dtype=dtype)
    
    # 2. Encode prompt
    do_classifier_free_guidance = guidance_scale > 1.0 or image_guidance_scale > 1.0
    prompt_embeds = pipeline_trained._encode_prompt(
        prompt,
        device,
        num_images_per_prompt=1,
        do_classifier_free_guidance=do_classifier_free_guidance,
        negative_prompt=None,
    )
    
    # 3. Prepare image latents (using pipeline's method which handles CFG expansion)
    batch_size = 1
    num_images_per_prompt = 1
    image_latents = pipeline_trained.prepare_image_latents(
        image,
        batch_size,
        num_images_per_prompt,
        prompt_embeds.dtype,
        device,
        do_classifier_free_guidance,
        generator=generator,
    )
    # image_latents is now [batch, batch, zeros_batch] if CFG, else [batch]
    
    # Get height/width from image latents (after CFG expansion, but dimensions are the same)
    # These are in latent space, need to convert to pixel space for prepare_latents
    height, width = image_latents.shape[-2:]
    height = height * pipeline_trained.vae_scale_factor
    width = width * pipeline_trained.vae_scale_factor
    
    # 4. Prepare timesteps
    pipeline_trained.scheduler.set_timesteps(num_inference_steps, device=device)
    timesteps = pipeline_trained.scheduler.timesteps
    
    # 5. Prepare noise latents (for the edited image)
    # Note: latents are NOT expanded here - they're expanded in the loop
    num_channels_latents = pipeline_trained.vae.config.latent_channels  # 4 channels
    latents = pipeline_trained.prepare_latents(
        batch_size * num_images_per_prompt,
        num_channels_latents,
        height,
        width,
        prompt_embeds.dtype,
        device,
        generator,
        latents=None,
    )
    
    # 6. Denoising loop with blended guidance (MEMORY OPTIMIZED)
    num_warmup_steps = len(timesteps) - num_inference_steps * pipeline_trained.scheduler.order
    
    # Note: Memory optimizations (attention slicing, VAE slicing) should be enabled
    # when loading the models, not here. This ensures they're set once.
    
    with torch.no_grad():  # Ensure no gradient computation
        with pipeline_trained.progress_bar(total=num_inference_steps) as progress_bar:
            for i, t in enumerate(timesteps):
                # Expand latents for classifier-free guidance (3-way)
                latent_model_input = torch.cat([latents] * 3) if do_classifier_free_guidance else latents
                latent_model_input = pipeline_trained.scheduler.scale_model_input(latent_model_input, t)
                
                # Concatenate image latents (InstructPix2Pix uses 8-channel input: 4 latents + 4 image)
                latent_model_input = torch.cat([latent_model_input, image_latents], dim=1)
                
                # Get noise predictions from both models (process sequentially to save memory)
                # Trained model prediction
                noise_pred_trained = pipeline_trained.unet(
                    latent_model_input,
                    t,
                    encoder_hidden_states=prompt_embeds,
                    return_dict=False,
                )[0]
                
                # Clear cache before second model call
                torch.cuda.empty_cache()
                
                # Pretrained model prediction
                noise_pred_pretrained = pipeline_pretrained.unet(
                    latent_model_input,
                    t,
                    encoder_hidden_states=prompt_embeds,
                    return_dict=False,
                )[0]
                
                # Blend predictions using omega (similar to bg_guidance.py)
                # omega=0.0: only pretrained, omega=1.0: only trained
                noise_pred = omega * noise_pred_trained + (1.0 - omega) * noise_pred_pretrained
                
                # Delete intermediate tensors to free memory
                del noise_pred_trained, noise_pred_pretrained
                torch.cuda.empty_cache()
                
                # Perform InstructPix2Pix-specific classifier-free guidance
                # InstructPix2Pix uses 3-way guidance: text, image, unconditional
                if do_classifier_free_guidance:
                    noise_pred_text, noise_pred_image, noise_pred_uncond = noise_pred.chunk(3)
                    noise_pred = (
                        noise_pred_uncond
                        + guidance_scale * (noise_pred_text - noise_pred_image)
                        + image_guidance_scale * (noise_pred_image - noise_pred_uncond)
                    )
                    del noise_pred_text, noise_pred_image, noise_pred_uncond
                
                # Compute the previous noisy sample x_t -> x_t-1
                latents = pipeline_trained.scheduler.step(noise_pred, t, latents, return_dict=False)[0]
                
                # Clean up
                del noise_pred, latent_model_input
                torch.cuda.empty_cache()
                
                # Update progress bar
                if i == len(timesteps) - 1 or ((i + 1) > num_warmup_steps and (i + 1) % pipeline_trained.scheduler.order == 0):
                    progress_bar.update()
    
    # 7. Decode latents to image
    with torch.no_grad():
        image = pipeline_trained.vae.decode(
            latents / pipeline_trained.vae.config.scaling_factor,
            return_dict=False
        )[0]
        # Detach to remove gradient requirement before postprocessing
        image = image.detach()
        image = pipeline_trained.image_processor.postprocess(image, output_type="pil")
    
    return image[0] if isinstance(image, list) else image


# ============================================================================
# Load models for background guidance
# ============================================================================

# Paths to the trained models
pretrained_model_path = "/content/drive/MyDrive/genai/instructpix2pix/pretrained_model"  # From Training 1
trained_model_path = "/content/drive/MyDrive/genai/instructpix2pix/trained_model"  # From Training 2

print("Loading models for background guidance mechanism...")
print(f"  - Pretrained model: {pretrained_model_path}")
print(f"  - Trained model: {trained_model_path}")

# Load pretrained pipeline (from stable-diffusion-v1-5)
print("\nLoading pretrained model (background/prior model)...")
pretrained_pipeline = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    pretrained_model_path, 
    torch_dtype=torch.float16
).to("cuda")

# Load trained pipeline (from customized fine-tuned SD model)
print("Loading trained model (fine-tuned model)...")
trained_pipeline = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    trained_model_path, 
    torch_dtype=torch.float16
).to("cuda")

# Enable memory optimizations for both pipelines
print("\nEnabling memory optimizations...")
pretrained_pipeline.enable_attention_slicing()
trained_pipeline.enable_attention_slicing()
pretrained_pipeline.enable_vae_slicing()
trained_pipeline.enable_vae_slicing()

# Optional: Use CPU offloading if memory is still an issue
# Uncomment the following lines if you still get OOM errors:
# print("Enabling CPU offloading (slower but uses less GPU memory)...")
# pretrained_pipeline.enable_model_cpu_offload()
# trained_pipeline.enable_model_cpu_offload()

print("\n✓ Both models loaded successfully!")

# Validate that both models are InstructPix2Pix models (8-channel UNet)
assert pretrained_pipeline.unet.config.in_channels == 8, \
    f"Pretrained model must be InstructPix2Pix (8 channels), got {pretrained_pipeline.unet.config.in_channels} channels"
assert trained_pipeline.unet.config.in_channels == 8, \
    f"Trained model must be InstructPix2Pix (8 channels), got {trained_pipeline.unet.config.in_channels} channels"
print("✓ Both models have compatible 8-channel UNet architecture")
print(f"  - Pretrained model: {pretrained_pipeline.unet.config.in_channels} channels")
print(f"  - Trained model: {trained_pipeline.unet.config.in_channels} channels")


In [ ]:
# ============================================================================
# Test the background guidance mechanism
# ============================================================================

print("\n=== Testing Background Guidance Method ===\n")
generator = torch.Generator("cuda").manual_seed(0)
image = load_image("https://huggingface.co/datasets/sayakpaul/sample-datasets/resolve/main/test_pix2pix_4.png")
prompt = "add duck on the lake"

# Test with different omega values to see the effect
omega_values = [0.0, 0.5, 1.0]
for omega in omega_values:
    print(f"\nGenerating with omega={omega} (omega=0.0: only pretrained, omega=1.0: only trained)...")
    edited_image = generate_with_bg_guidance(
        pipeline_trained=trained_pipeline,
        pipeline_pretrained=pretrained_pipeline,
        prompt=prompt,
        image=image,
        num_inference_steps=20,
        image_guidance_scale=2.0,
        guidance_scale=8.0,
        omega=omega,
        generator=generator,
    )
    
    filename = f"duck_omega_{omega:.1f}.png"
    edited_image.save(filename)
    print(f"Saved: {filename}")
    
    # Display the image
    display(edited_image)
